# Récupération ETI31 et premier nettoyage de données

In [72]:
import pickle

In [1]:
#Importation
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn as sk
from sklearn import tree
import matplotlib.pyplot as plt
from textblob import TextBlob
from statistics import mean
from math import ceil
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import graphviz
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [2]:
import re

Un premier travail de nettoyage a été fait sur le dataset : Ne garder que les colonnes de commentaires et qui permettent l'identification.

Cela a permis de réduire drastiquement le nombre de colonnes de plus de 200 à 8.

Un seconde travail a été de nettoyer les lignes ne contenant aucun commentaire puisque celles-ci seront inutiles pour l'analyse de texte. On est passé d'environ 5800 lignes à environ 4200.

In [98]:
#Chargement des données
path_eti31 = "CSV/ETI31_commentaires.csv"
eti31 = pd.read_csv(path_eti31,low_memory=False)

In [99]:
eti31

,No DESIGNATION,COMMENTAIRE DE SIG,CODE RELEVE,COMMENTAIRE,BLOC NOTE,COMMLITIGEPIDI
0,299568158,NaN,RRC,# Obtenu MARCHAND CHRISTOPHE le 26/04/2023 a 1...,NaN,\
1,297892193,NaN,DMS,/DMS/#PTO_existante#-#Malfa on PLP non detect ...,NaN,\\$GM$ Un rdv a ete repris par le technicien p...
2,297897767,NaN,DMS,/DMS/#PTO_non existante#/Mohand ok raccordemen...,NaN,\\$GM$ CP Obtenu cliente le 13/04 Le client a...
3,223370510,NaN,DMS,/DMS/#PTO_non existante#/Installation complete...,NaN,\
4,299556603,NaN,DMS,/DMS/#PTO_existante#-#Malfa on PLP non detect ...,NaN,\
...,...,...,...,...,...,...
5028,296611510,NaN,DMS,/DMS/#PTO_existante#-#Malfacon PLP non detecte...,NaN,\
5029,223273821,NaN,NaN,NaN,$annule otomate #549$,NaN
5030,223309738,NaN,DMS,/DMS/#PTO_non existante#/Raccordement effectue...,NaN,\
5031,298170718,NaN,DMS,/DMS/#PTO_non existante#/Raccordement en chamb...,NaN,\\$GM$ Obtenu M. Couchevellou au 0658359425 le...


In [100]:
eti31.columns

Index(['No DESIGNATION', 'COMMENTAIRE DE SIG', 'CODE RELEVE', 'COMMENTAIRE',
       'BLOC NOTE', 'COMMLITIGEPIDI'],
      dtype='object')

# 1 - Le nettoyage des données et le formatage du dataframe

In [101]:
eti31 = eti31.dropna(subset=['CODE RELEVE',])
eti31 = eti31.dropna(subset=["COMMENTAIRE DE SIG","COMMENTAIRE","BLOC NOTE","COMMLITIGEPIDI"],how='all')
eti31 = eti31.reset_index(drop=True)

In [102]:
eti31

,No DESIGNATION,COMMENTAIRE DE SIG,CODE RELEVE,COMMENTAIRE,BLOC NOTE,COMMLITIGEPIDI
0,299568158,NaN,RRC,# Obtenu MARCHAND CHRISTOPHE le 26/04/2023 a 1...,NaN,\
1,297892193,NaN,DMS,/DMS/#PTO_existante#-#Malfa on PLP non detect ...,NaN,\\$GM$ Un rdv a ete repris par le technicien p...
2,297897767,NaN,DMS,/DMS/#PTO_non existante#/Mohand ok raccordemen...,NaN,\\$GM$ CP Obtenu cliente le 13/04 Le client a...
3,223370510,NaN,DMS,/DMS/#PTO_non existante#/Installation complete...,NaN,\
4,299556603,NaN,DMS,/DMS/#PTO_existante#-#Malfa on PLP non detect ...,NaN,\
...,...,...,...,...,...,...
4183,223275005,NaN,TVC,NaN,Merci de cleturer le rendez vous du client car...,\\$GM$ Un rdv a ete repris par le tehcnicien p...
4184,296611510,NaN,DMS,/DMS/#PTO_existante#-#Malfacon PLP non detecte...,NaN,\
4185,223309738,NaN,DMS,/DMS/#PTO_non existante#/Raccordement effectue...,NaN,\
4186,298170718,NaN,DMS,/DMS/#PTO_non existante#/Raccordement en chamb...,NaN,\\$GM$ Obtenu M. Couchevellou au 0658359425 le...


In [103]:
eti31.to_pickle("PICKLE/eti31.pkl")

## 2 - Labelisation des échecs et réussites :
Je fais le choix de passer par des booléens pour plus de facilité. True correspond à une réussite et False à un échec.

In [104]:
label_echec = ["ANC","ANN","ETU","MAJ","ORT","PAD","PBC","REO","RMC","RMF","RRC"]

Code client : ANN, PAD, RMC

In [105]:
eti31["COMMENTAIRE DE SIG"].unique() #On la garde quand même au cas où de futur fichiers auraient des données dans cette colonne

array([nan])

In [106]:
df = eti31
def colonne_label():
    colonne = []
    for i in df["CODE RELEVE"]:
        if i in label_echec:
            colonne.append(False)
        else :
            colonne.append(True)
    return colonne

In [107]:
df["Reussite"]=colonne_label() 
df = df.drop(columns="CODE RELEVE")

In [108]:
df

,No DESIGNATION,COMMENTAIRE DE SIG,COMMENTAIRE,BLOC NOTE,COMMLITIGEPIDI,Reussite
0,299568158,NaN,# Obtenu MARCHAND CHRISTOPHE le 26/04/2023 a 1...,NaN,\,False
1,297892193,NaN,/DMS/#PTO_existante#-#Malfa on PLP non detect ...,NaN,\\$GM$ Un rdv a ete repris par le technicien p...,True
2,297897767,NaN,/DMS/#PTO_non existante#/Mohand ok raccordemen...,NaN,\\$GM$ CP Obtenu cliente le 13/04 Le client a...,True
3,223370510,NaN,/DMS/#PTO_non existante#/Installation complete...,NaN,\,True
4,299556603,NaN,/DMS/#PTO_existante#-#Malfa on PLP non detect ...,NaN,\,True
...,...,...,...,...,...,...
4183,223275005,NaN,NaN,Merci de cleturer le rendez vous du client car...,\\$GM$ Un rdv a ete repris par le tehcnicien p...,True
4184,296611510,NaN,/DMS/#PTO_existante#-#Malfacon PLP non detecte...,NaN,\,True
4185,223309738,NaN,/DMS/#PTO_non existante#/Raccordement effectue...,NaN,\,True
4186,298170718,NaN,/DMS/#PTO_non existante#/Raccordement en chamb...,NaN,\\$GM$ Obtenu M. Couchevellou au 0658359425 le...,True


In [110]:
df.to_pickle("PICKLE/dataframe_labelise.pkl")